# Hunga Tonga-Hunga Ha'apa Volcano-generated Tsunami, January 15, 2022

Let's now focus on the eruption, as we initially planned!

***

Sadly, there is yet, no available tsunami propagation model for the Tonga eruption that can be analysed for now...


**But we can use some of the monitoring system available in the Pacific Ocean!**


Specifically, we can take advantage of the DART® monitoring system:

DART® are ocean-bottom pressure sensors, able to measure tsunamis in the open ocean, are providing important data on tsunami propagation in deep water, and satellite communications have enabled the data to be used in real time to detect and measure tsunami waves in the deep ocean. 


<img src="https://nctr.pmel.noaa.gov/images/DART_locationsbyowner.png" width="80%">

DART® real-time tsunami monitoring systems are positioned at strategic locations throughout the ocean and play a critical role in tsunami forecasting.

**More info: https://nctr.pmel.noaa.gov/Dart/**

The dataset is available for various tsunami events and luckily the one for the Hunga Tonga eruption could be downloaded from:

**https://www.ngdc.noaa.gov/hazel/view/hazards/tsunami/related-runups/5824**

In [ ]:
import pandas as pd
import numpy as np

import cmocean

from IPython.display import YouTubeVideo

import scipy.stats as stats
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings('ignore', category=FutureWarning) 
warnings.filterwarnings('ignore', category=UserWarning)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['mathtext.fontset'] = 'cm'


def display_yotube_video(url, **kwargs):
    """
    Displays a Youtube video in a Jupyter notebook.
    
    Args:
        url (string): a link to a Youtube video.
        **kwargs: further arguments for IPython.display.YouTubeVideo
    
    Returns:
        YouTubeVideo: a video that is displayed in your notebook.
    """
    id_ = url.split("=")[-1]
    return YouTubeVideo(id_, **kwargs)

## Loading the dataset

I have already uploaded the dataset to the environment and we will open it using the `pandas` library:

In [ ]:
dataframe = pd.read_csv("../../dataset/runups-2022-02-24_16-23-00_+1100.tsv", sep='\t')
dataframe.drop('Unnamed: 0', axis=1, inplace=True)
dataframe = dataframe[dataframe['Maximum Water Height (m)'].notna()]
dataframe.loc[0,'Max Wave Arrival Day'] = 15.
dataframe.loc[0,'Max Wave Arrival Hour'] = 4.
dataframe.loc[0,'Max Wave Arrival Minute'] = 27. 
dataframe

Let's define the time in minutes since the start of the eruption and call it `julian`

In [ ]:
dataframe['julian'] = dataframe['Max Wave Arrival Day']*24*60.
dataframe['julian'] += dataframe['Max Wave Arrival Hour']*60. 
dataframe['julian'] += dataframe['Max Wave Arrival Minute'] 
dataframe['julian'] -= dataframe.loc[0,'julian']

We will also clean the dataset to only keep what we will need for our analysis:

In [ ]:
df = dataframe.drop('Initial Wave Arrival Day', axis=1)
df.drop('Initial Wave Arrival Hour', axis=1, inplace=True)
df.drop('Initial Wave Arrival Minute', axis=1, inplace=True)
df.drop('Travel Hours', axis=1, inplace=True)
df.drop('Travel Minutes', axis=1, inplace=True)
df.drop('Max Inundation Distance (m)', axis=1, inplace=True)
df.drop('Max Wave Arrival Day', axis=1, inplace=True)
df.drop('Max Wave Arrival Hour', axis=1, inplace=True)
df.drop('Max Wave Arrival Minute', axis=1, inplace=True)
df

## Visualising the DART dataset

We will plot the following:
    
+ Maximum water height
+ Distance from eruption
+ Travel time


Let's start with the water height:

In [ ]:
lons = df.Longitude.values
lats = df.Latitude.values


travelTime = df['julian'].values
dist = df['Distance From Source (km)'].values
maxWaterHeight = df['Maximum Water Height (m)'].values

In [ ]:
proj = ccrs.Mollweide(180)

fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection=proj)
ax.stock_img()
ax.coastlines(linewidth=1)

sz = maxWaterHeight.copy()
sz[sz>1.5] = 1.5

pts = ax.scatter(lons, lats, s=90*sz, c=maxWaterHeight, cmap=cmocean.cm.balance,
           edgecolor='black',alpha=0.75, transform=ccrs.PlateCarree(),vmin=0,vmax=1.5)

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065, 
                    orientation="horizontal")
cbar.set_label('Maximum water height (m)', rotation=0, 
               labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.show()
fig.clear()
plt.close(fig)

Now the distance from the eruption:

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection=proj)
ax.stock_img()
ax.coastlines(linewidth=1)

pts = ax.scatter(lons, lats, s=dist/200., c=dist, cmap=cmocean.cm.deep,
           edgecolor='black',alpha=0.75, transform=ccrs.PlateCarree(), vmin=3000,vmax=15000)

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065, 
                    orientation="horizontal")
cbar.set_label('Distance from eruption (km)', rotation=0, 
               labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.show()
fig.clear()
plt.close(fig)

And the travel time:

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection=proj)
ax.stock_img()
ax.coastlines(linewidth=1)

pts = ax.scatter(lons, lats, s=travelTime/10., c=travelTime, cmap=cmocean.cm.thermal,
           edgecolor='black',alpha=0.75, transform=ccrs.PlateCarree())

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065, 
                    orientation="horizontal")
cbar.set_label('Travel time (mins)', rotation=0, 
               labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.show()
fig.clear()
plt.close(fig)

From this dataset we can infer the speed of the tsunami wave at each location:

In [ ]:
### Compute the tsunami speed
df['speed'] = df['Distance From Source (km)']*1000./(df['julian']*60.)
vel = df['speed'].values

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection=proj)
ax.stock_img()
ax.coastlines(linewidth=1)

pts = ax.scatter(lons, lats, s=vel/2., c=vel, cmap=cmocean.cm.speed,
           edgecolor='black',alpha=0.75, transform=ccrs.PlateCarree(), vmin=100,vmax=300)

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065, 
                    orientation="horizontal")
cbar.set_label('Tsunami speed (m/s)', rotation=0, 
               labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.show()
fig.clear()
plt.close(fig)

## Graphs

Looking at the data globally on a map is great but let's see the different trends. To do so we will take all the dataset and perform some linear regression fitting on them to evaluate if there is any relationships that emerge...

In [ ]:
def regressionFitting(x, y):
    slope, intercept = np.polyfit(x, y, 1)  # linear model adjustment

    y_model = np.polyval([slope, intercept], x)   # modeling...

    x_mean = np.mean(x)
    y_mean = np.mean(y)
    n = x.size                        # number of samples
    m = 2                             # number of parameters
    dof = n - m                       # degrees of freedom
    t = stats.t.ppf(0.975, dof)       # Students statistic of interval confidence

    residual = y - y_model

    std_error = (np.sum(residual**2) / dof)**.5   # Standard deviation of the error

    # calculating the r2
    # https://www.statisticshowto.com/probability-and-statistics/coefficient-of-determination-r-squared/
    # Pearson's correlation coefficient
    numerator = np.sum((x - x_mean)*(y - y_mean))
    denominator = ( np.sum((x - x_mean)**2) * np.sum((y - y_mean)**2) )**.5
    correlation_coef = numerator / denominator
    r2 = correlation_coef**2

    # mean squared error
    MSE = 1/n * np.sum( (y - y_model)**2 )

    # to plot the adjusted model
    x_line = np.linspace(np.min(x), np.max(x), 100)
    y_line = np.polyval([slope, intercept], x_line)

    # confidence interval
    ci = t * std_error * (1/n + (x_line - x_mean)**2 / np.sum((x - x_mean)**2))**.5
    
    print('Linear regression results:')
    a = str(np.round(intercept))
    b = str(np.round(slope,5))
    r2s = str(np.round(r2,5))
    MSEs = str(np.round(MSE,2))
    print('   y = ' + a + ' + ' + b + ' x')
    print('   r2 = ' + r2s + '     MSE = ' + MSEs)

    return x_line, y_line, ci

+ For the maximum water height

In [ ]:
x_line, y_line, ci = regressionFitting(dist, maxWaterHeight)

fig = plt.figure(figsize=(10,4))
ax = plt.gca()
pts = plt.scatter(dist, maxWaterHeight, s=90*sz, c=maxWaterHeight, cmap=cmocean.cm.haline_r,
           edgecolor='black',alpha=0.75, vmin=0,vmax=1.5, zorder=4)
ax.plot(x_line, y_line, color = 'k', lw=2, zorder=2)
ax.fill_between(x_line, y_line + ci, y_line - ci, color = 'lightgrey', 
                label = '95% confidence interval', zorder=1)
ax.set_xlabel('Distance (km)',fontsize=8)
ax.set_ylabel('Maximum water height (m)',fontsize=8)
ax.set_xlim(0,13000)
ax.set_ylim(0,2.4)

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065)
cbar.set_label('Maximum water height (m)', labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.legend(fontsize=8)
plt.show()
fig.clear()
plt.close(fig)

+ For the Tsunami speed 

In [ ]:
ids = np.where(~np.isnan(vel))[0]
x_line, y_line, ci = regressionFitting(dist[ids], vel[ids])

fig = plt.figure(figsize=(10,4))
ax = plt.gca()

pts = plt.scatter(dist, vel, s=vel/2., c=vel, cmap=cmocean.cm.speed,
           edgecolor='black',alpha=0.75, vmin=100, vmax=300, zorder=4)
ax.plot(x_line, y_line, color = 'k', lw=2, zorder=2)
ax.fill_between(x_line, y_line + ci, y_line - ci, color = 'lightgrey', 
                label = '95% confidence interval', zorder=1)
ax.set_xlabel('Distance (km)',fontsize=8)
ax.set_ylabel('Tsunami speed (m/s)',fontsize=8)
ax.set_xlim(0,13100)
ax.set_ylim(0,500)

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065)
cbar.set_label('Tsunami speed (m/s)', labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.legend(fontsize=8)
plt.show()
fig.clear()
plt.close(fig)

We can also use the period and based on the speed compute the wavelenght of the Tsunami:

In [ ]:
df['wavelenght'] = df['speed']*df['Period']*60/1000.

In [ ]:
wavelenght = df['wavelenght'].values
ids = np.where(~np.isnan(wavelenght))[0]
x_line, y_line, ci = regressionFitting(dist[ids], wavelenght[ids])

fig = plt.figure(figsize=(10,4))
ax = plt.gca()

pts = plt.scatter(dist, wavelenght, s=wavelenght/5, c=wavelenght, cmap=cmocean.cm.tempo,
           edgecolor='black',alpha=0.75, vmin=100,vmax=300, zorder=4)
ax.plot(x_line, y_line, color = 'k', lw=2, zorder=2)
ax.fill_between(x_line, y_line + ci, y_line - ci, color = 'lightgrey', 
                label = '95% confidence interval', zorder=1)
ax.set_xlabel('Distance (km)',fontsize=8)
ax.set_ylabel('Wavelenght (km)',fontsize=8)
ax.set_xlim(0,13100)
ax.set_ylim(0,800)

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065)
cbar.set_label('Wavelenght (km)', labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.legend(fontsize=8)
plt.show()
fig.clear()
plt.close(fig)

## Building a simple tsunami propagation model

Assuming **Airy velocity**,  a wave is characterized as a shallow-water wave when the ratio of the water depth and wavelength is very small. 

The velocity of a shallow-water wave is also equal to the square root of the product of the acceleration of gravity and the depth of the water.  Here the wavelenght $\lambda$ is about 200 km so $\lambda/20$ is approximately 10 km, meaning that we could consider that the tsunami velocity is of the form:

$$ v = \sqrt{g h}$$

where $g$ is the gravity and $h$ the water depth. 

We could get an estimate of the average water depth based on the Tsunami speed:

In [ ]:
g = 9.81
df['depth'] = df['speed']**2/g
depth = df['depth'].values/1000.

In [ ]:
ids = np.where(~np.isnan(depth))[0]
x_line, y_line, ci = regressionFitting(dist[ids], depth[ids])

fig = plt.figure(figsize=(10,4))
ax = plt.gca()

pts= plt.scatter(dist, depth, s=depth*10, c=depth, cmap=cmocean.cm.deep,
           edgecolor='black',alpha=0.75, vmin=0, vmax=10, zorder=4)
ax.plot(x_line, y_line, color = 'k', lw=2, zorder=2)
ax.fill_between(x_line, y_line + ci, y_line - ci, color = 'lightgrey', 
                label = '95% confidence interval', zorder=1)
ax.set_xlabel('Distance (km)',fontsize=8)
ax.set_ylabel('Approximated water depth (km)',fontsize=8)
ax.set_xlim(0,13100)
ax.set_ylim(0,10)

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065)
cbar.set_label('Approximated water depth (km)', labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.legend(fontsize=8)
plt.show()
fig.clear()
plt.close(fig)

Using the equation above $ v = \sqrt{g h}$, we could also compute a first-order estimate of the tsunami front wave evolution, for example let's take the average depth of the Pacific Ocean (4 km). We can then compute the tsunami front for every 4 hours interval.

Also we will compute the front of the sound propagation (assuming a velocity for the sound of 343 m/s).

In [ ]:
hours = 4
average_depth = 4000.

lon, lat = dataframe.loc[0,'Longitude'], dataframe.loc[0,'Latitude']

velocity = np.sqrt(average_depth * g)*60.
sound = 343.0

hours2min = hours*60
timeTravel = np.arange(0,1300,hours2min)
soundTravel = np.zeros(len(timeTravel))
distTravel = np.zeros(len(timeTravel))
for k in range(1,len(timeTravel)):
    distTravel[k] = velocity*timeTravel[k]/1000.
    soundTravel[k] = sound*60.*timeTravel[k]/1000.

Let's plot the position of the tsunami and explosion sound over time:

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""
    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection=proj)
ax.stock_img()
ax.coastlines(linewidth=1)

# setup a Normalization instance
norm = colors.Normalize(distTravel.min(),distTravel.max())

# define the colormap
cmap = discrete_cmap(len(distTravel), cmocean.cm.matter)

# Use the norm and cmap to define the edge colours
edgecols = cmap(norm(distTravel))

for k in range(1,len(distTravel)):
    ax.tissot(rad_km=distTravel[k], lons=lon, lats=lat, 
              facecolor="None", edgecolor=edgecols[k], 
              linestyle='-',linewidth=5, alpha = 0.95, 
              label=str(k*hours)+'h'
             )
    ax.tissot(rad_km=soundTravel[k], lons=lon, lats=lat, 
              facecolor="None", edgecolor=edgecols[k], 
              linestyle='--',linewidth=2, alpha = 0.95, 
              label=str(k*hours)+'h'
             )
    plt.gcf().text(0.415+(k-1)*0.05, 0.25, str(k*hours)+'h', 
                   weight="bold", 
                   c=edgecols[k], fontsize=10)
    
pts = ax.scatter(lon, lat, s=50, c='w', 
           edgecolor='black',alpha=1, transform=ccrs.PlateCarree(), zorder=50)


plt.show()
fig.clear()
plt.close(fig)

We can look at how our model performs against DART dataset:

In [ ]:
df['simTravelTime Minute'] = df['Distance From Source (km)']*1000./velocity
simTime = df['simTravelTime Minute'].values

fig = plt.figure(figsize=(8,6.5))
ax = plt.gca()

pts = plt.scatter(travelTime, simTime, s=dist/100., c=dist, cmap=cmocean.cm.tempo,
           edgecolor='black',alpha=0.75, zorder=4, vmin=0, vmax=dist.max(), )
ax.plot(travelTime, travelTime, lw=5, color = 'lightgrey', 
        zorder=1)
ax.plot(travelTime, travelTime, lw=1.5, color = 'k', label = 'Best-fit line', zorder=2)
ax.set_xlabel('Observed travel time (minutes)',fontsize=8)
ax.set_ylabel('Simulated travel time (minutes)',fontsize=8)
ax.set_xlim(0,1500)
ax.set_ylim(0,1500)

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065)
cbar.set_label('Distance to eruption (km)', labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.legend(fontsize=8)
plt.show()
fig.clear()
plt.close(fig)


ids = np.where(~np.isnan(travelTime))[0]
x_line, y_line, ci = regressionFitting(travelTime[ids], simTime[ids])

***

Obvisouly our model is really simple and the match with observations is not great! 

<div class="alert alert-block alert-info">Could you think of other things that we might add to help getting a better model?</div>



In [ ]:
display_yotube_video("https://www.youtube.com/watch?v=_fvqWqfrXXM", width=800, height=500)

## Sound wave evaluation

![alt text](https://petapixel.com/assets/uploads/2022/01/tonga-volcano-eruption-goes-close.gif)


Here we will use some observations I found on the web (twitter, online journals) reporting people hearing the sound of the eruption.

+ _The eruption was heard more than 2300 km away in New Zealand, where the sound arrived two hours later._
+ _A series of bangs were heard around 3:30 a.m. local time in and around Anchorage, Alaska, approximately 9700 km away from the volcano._
+ _The shockwave was traveling almost as fast as the speed of sound and causing noticeable jumps in atmospheric pressure as it reached the UK on the opposite side of the world (16146 km) about 15 hours after the eruption._

For Alaska, we need to use the UTC time to get the time it took for the sound to travel. The eruption was at 4:27 a.m. UTC time and it was at 3:30 am Alaska time (-9h UTC) meaning that it was 12:30 pm in UTC time. So the difference was about 8h.

Let's evaluate the sound velocity based on these observations:

In [ ]:
NZlat, NZlon = -41.2924, -185.3
velNZ = 2300*1000/(2*3600)

Alaslat, Alaslon = 61.2181, -150.1003
velAla = 9700*1000/(8*3600)

UKlat, UKlon = 51.5072, 0.1276
velUK = 16146*1000/(15*3600)

print('Estimated velocities from observations:')
print('   + NZ     ',round(velNZ,1),' m/s')
print('   + Alaska ',round(velAla,1),' m/s')
print('   + UK     ',round(velUK,1),' m/s')

We can plot the difference between observed and modelled sound wave position for these different locations:

In [ ]:
fig = plt.figure(figsize=(12,10))


ax = plt.axes(projection=proj)
ax.stock_img()
ax.coastlines(linewidth=1)

pts = ax.scatter(lon, lat, s=50, c='blue', 
                 edgecolor='black',alpha=1, 
                 transform=ccrs.PlateCarree(), zorder=50)

# New Zealand          
plt.plot([lon, NZlon], [lat, NZlat], ls='--', lw=2, 
         color='w', alpha = 0.75, 
         transform=ccrs.PlateCarree())

pts = ax.scatter(NZlon, NZlat, s=50, c='w', 
                 edgecolor='w',alpha=1, 
                 transform=ccrs.PlateCarree(), zorder=50)

ax.tissot(rad_km=2300, lons=lon, lats=lat, 
          facecolor="None", edgecolor='ww', 
          linestyle='--',linewidth=3, alpha = 1,
         )
ax.tissot(rad_km=2376, lons=lon, lats=lat, 
          facecolor="None", edgecolor='k', 
          linestyle='--',linewidth=1, alpha = 1,
         )

# Alaska
pts = ax.scatter(Alaslon, Alaslat, s=50, c='orange', 
                 edgecolor='w',alpha=1, 
                 transform=ccrs.PlateCarree(), zorder=50)

plt.plot([lon, Alaslon], [lat, Alaslat], ls='--', lw=2, 
         color='orange', alpha = 0.75,
         transform=ccrs.PlateCarree())

ax.tissot(rad_km=9500, lons=lon, lats=lat, 
          facecolor="None", edgecolor='orange', 
          linestyle='-',linewidth=3, alpha = 1, 
         )

ax.tissot(rad_km=9504, lons=lon, lats=lat, 
          facecolor="None", edgecolor='k', 
          linestyle='--',linewidth=1, alpha = 1, 
         )


# UK
pts = ax.scatter(UKlon, UKlat, s=50, c='red', 
                 edgecolor='k',alpha=1, 
                 transform=ccrs.PlateCarree(), zorder=50)

plt.plot([lon, UKlon], [lat, UKlat], ls='--', lw=2, 
         color='red', alpha = 0.75, 
         transform=ccrs.PlateCarree())

ax.tissot(rad_km=16146, lons=lon, lats=lat, 
          facecolor="None", edgecolor='red', 
          linestyle='-',linewidth=3, alpha = 1., 
         )

ax.tissot(rad_km=17820, lons=lon, lats=lat, 
          facecolor="None", edgecolor='k', 
          linestyle='--',linewidth=1, alpha = 1, 
         )

plt.show()
fig.clear()
plt.close(fig)

These predictions from our model seem to work much better for the sound wave!!!